### **Efficient Yelp API Calls**

In [1]:
# Standard Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Additional Imports
import os, json, math, time
from yelpapi import YelpAPI
from tqdm.notebook import tqdm_notebook

### **Credentials and Accessing the API**

In [2]:
# Load API Credentials
with open('/Users/Erfoxcroft/.secret/yelp_api.json') as f:   #use your path here!
    login = json.load(f)
# Instantiate YelpAPI Variable
yelp_api = YelpAPI(login['api-key'], timeout_s=5.0)

### **Define Search**

- Use the Yelp API to search your favorite city for a cuisine type of your choice <br>
- Extract all of the results from your search and compile them into one dataframe using a for loop

In [3]:
# Set our API call parameters 
LOCATION = 'NY,NY'
TERM = 'seafood' # Search Term #

### **Create a JSON file where your results will be saved**

In [4]:
# Specifying JSON_FILE filename (can include a folder)
# include the search terms in the filename
JSON_FILE = "Data/results_in_progress_NY_seafood.json"
JSON_FILE

'Data/results_in_progress_NY_seafood.json'

- **Create a results-in-progress JSON file, but only if it doesn't exist**

In [5]:
## Check if JSON_FILE exists
file_exists = os.path.isfile(JSON_FILE)
## If it does not exist: 
if file_exists == False:
    
    ## CREATE ANY NEEDED FOLDERS
    # Get the Folder Name only
    folder = os.path.dirname(JSON_FILE)
    ## If JSON_FILE included a folder:
    if len(folder)>0:
        # create the folder
        os.makedirs(folder,exist_ok=True)
        
        
    ## INFORM USER AND SAVE EMPTY LIST
    print(f'[i] {JSON_FILE} not found. Saving empty list to file.')
    
    
    # save an empty list
    with open(JSON_FILE,'w') as f:
        json.dump([],f)  
# If it exists, inform user
else:
    print(f"[i] {JSON_FILE} already exists.")

[i] Data/results_in_progress_NY_seafood.json not found. Saving empty list to file.


- **Determine how many results are already in the file**

In [6]:
## Load previous results and use len of results for offset
with open(JSON_FILE,'r') as f:
    previous_results = json.load(f)
    
## set offset based on previous results
n_results = len(previous_results)
print(f'- {n_results} previous results found.')

- 0 previous results found.


- **Decide how many pages of results will be needed**

In [7]:
# use our yelp_api variable's search_query method to perform our API call
results = yelp_api.search_query(location=LOCATION,
                                term=TERM,
                               offset=n_results)
results.keys()

dict_keys(['businesses', 'total', 'region'])

In [8]:
## How many results total?
total_results = results['total']
total_results

13200

In [9]:
## How many did we get the details for?
results_per_page = len(results['businesses'])
results_per_page

20

In [10]:
# Use math.ceil to round up for the total number of pages of results.
n_pages = math.ceil((results['total']-n_results)/ results_per_page)
n_pages

660

- **Add page of results to .JSON file**

In [11]:
# join new results with old list with extend and save to file
previous_results.extend(results['businesses'])  
with open(JSON_FILE,'w') as f:
     json.dump(previous_results,f)

### **Create a Function**

In [12]:
def create_json_file(JSON_FILE, delete_if_exists=False):
    # Check if JSON_FILE exists
    file_exists = os.path.isfile(JSON_FILE)

    # If it DOES exist:
    if file_exists == True:
        # Check if user wants to delete if exists
        if delete_if_exists:
            print(f"[!] {JSON_FILE} already exists. Deleting previous file...")
            # delete file
            os.remove(JSON_FILE)
            # Recursive call to function after old file deleted
            create_json_file(JSON_FILE, delete_if_exists=False)
        else:
            print(f"[i] {JSON_FILE} already exists.")
    
    # If it does NOT exist:
    else:
        # INFORM USER AND SAVE EMPTY LIST
        print(f"[i] {JSON_FILE} not found. Saving empty list to new file.")

        # CREATE ANY NEEDED FOLDERS
        # Get the Folder Name only
        folder = os.path.dirname(JSON_FILE)

        # If JSON_FILE included a folder:
        if len(folder) > 0:
            # create the folder
            os.makedirs(folder, exist_ok=True)
        
        # Save empty list to start the json file
        with open(JSON_FILE, 'w') as f:
            json.dump([], f)

# Example usage:
# create_json_file("example.json", delete_if_exists=True)

In [13]:
## Create a new empty json file (delete the previous if it exists)
create_json_file(JSON_FILE, delete_if_exists=True)
## Load previous results and use len of results for offset
with open(JSON_FILE,'r') as f:
    previous_results = json.load(f)
    
## set offset based on previous results
n_results = len(previous_results)
print(f'- {n_results} previous results found.')
# use our yelp_api variable's search_query method to perform our API call
results = yelp_api.search_query(location=LOCATION,
                                term=TERM,
                               offset=n_results)
## How many results total?
total_results = results['total']
## How many did we get the details for?
results_per_page = len(results['businesses'])
# Use math.ceil to round up for the total number of pages of results.
n_pages = math.ceil((results['total']-n_results)/ results_per_page)
n_pages

[!] Data/results_in_progress_NY_seafood.json already exists. Deleting previous file...
[i] Data/results_in_progress_NY_seafood.json not found. Saving empty list to new file.
- 0 previous results found.


645

In [14]:
for i in tqdm_notebook( range(1,n_pages+1)):
    
    ## Read in results in progress file and check the length
    with open(JSON_FILE, 'r') as f:
        previous_results = json.load(f)
    ## save number of results for to use as offset
    n_results = len(previous_results)
    
    if (n_results + results_per_page) > 1000:
        print('Exceeded 1000 api calls. Stopping loop.')
        break## use n_results as the OFFSET 
    results = yelp_api.search_query(location=LOCATION,
                                    term=TERM, 
                                    offset=n_results)
    
    
    
    ## append new results and save to file
    previous_results.extend(results['businesses'])
    
    # display(previous_results)
    with open(JSON_FILE,'w') as f:
        json.dump(previous_results,f)
    
    time.sleep(.2)

# Exceeded 1000 api calls. Stopping loop.

  0%|          | 0/645 [00:00<?, ?it/s]

Exceeded 1000 api calls. Stopping loop.


In [16]:
# load final results
final_df = pd.read_json(JSON_FILE)
display(final_df.head(), final_df.tail())

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
0,YvWQMlMjtYmVmheH97tTlQ,crab-house-all-you-can-eat-seafood-new-york,Crab House All You Can Eat Seafood,https://s3-media2.fl.yelpcdn.com/bphoto/1J0qtJ...,False,https://www.yelp.com/biz/crab-house-all-you-ca...,936,"[{'alias': 'buffets', 'title': 'Buffets'}, {'a...",4.5,"{'latitude': 40.759664, 'longitude': -73.969776}","[pickup, delivery]",$$$$,"{'address1': '135 E 55th St', 'address2': None...",+12129330264,(212) 933-0264,6381.672940
1,yrH_D2YMYIydwKk8MrVlqA,crab-house-brooklyn-all-you-can-eat-seafood-br...,Crab House Brooklyn All You Can Eat Seafood,https://s3-media4.fl.yelpcdn.com/bphoto/nLkqUV...,False,https://www.yelp.com/biz/crab-house-brooklyn-a...,103,"[{'alias': 'seafood', 'title': 'Seafood'}, {'a...",4.5,"{'latitude': 40.57594204454698, 'longitude': -...",[],$$$$,"{'address1': '1223 Surf Ave', 'address2': None...",+13474621934,(347) 462-1934,14446.828706
2,LpVbI-E9C-CSp7WhU5h2Iw,fishmarket-restaurant-new-york,Fishmarket Restaurant,https://s3-media1.fl.yelpcdn.com/bphoto/-rU1vu...,False,https://www.yelp.com/biz/fishmarket-restaurant...,636,"[{'alias': 'seafood', 'title': 'Seafood'}, {'a...",4.5,"{'latitude': 40.70717, 'longitude': -74.00192}",[delivery],$$,"{'address1': '111 South St', 'address2': '', '...",+19173638101,(917) 363-8101,667.940697
3,Ch_m0ZPPzFJF7oi6dyk1Yw,crab-shack-seafood-boil-queens,Crab Shack Seafood Boil,https://s3-media3.fl.yelpcdn.com/bphoto/7Op8-z...,False,https://www.yelp.com/biz/crab-shack-seafood-bo...,28,"[{'alias': 'seafood', 'title': 'Seafood'}, {'a...",4.5,"{'latitude': 40.7006229, 'longitude': -73.8359...","[pickup, delivery, restaurant_reservation]",NaN,"{'address1': '5525 Myrtle Ave', 'address2': No...",+13476899949,(347) 689-9949,7330.684086
4,hdiuRS9sVZSMReZm4oV5SA,da-andrea-new-york,Da Andrea,https://s3-media1.fl.yelpcdn.com/bphoto/OciLdd...,False,https://www.yelp.com/biz/da-andrea-new-york?ad...,2105,"[{'alias': 'italian', 'title': 'Italian'}, {'a...",4.5,"{'latitude': 40.736218, 'longitude': -73.99597}","[pickup, delivery, restaurant_reservation]",$$,"{'address1': '35 W 13th St', 'address2': '', '...",+12123671979,(212) 367-1979,3424.729337


,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
995,lxt9V-AT3UvxQVNZVj45qg,prode-little-neck,Prode,https://s3-media2.fl.yelpcdn.com/bphoto/L7FNfK...,False,https://www.yelp.com/biz/prode-little-neck?adj...,100,"[{'alias': 'thai', 'title': 'Thai'}, {'alias':...",5.0,"{'latitude': 40.769369, 'longitude': -73.736158}","[delivery, pickup, restaurant_reservation]",NaN,"{'address1': '251-28 Northern Blvd', 'address2...",+17183049744,(718) 304-9744,22880.650025
996,uVfOB37NyYEGRs5FITM-nQ,leyenda-brooklyn,Leyenda,https://s3-media2.fl.yelpcdn.com/bphoto/xmzICv...,False,https://www.yelp.com/biz/leyenda-brooklyn?adju...,323,"[{'alias': 'latin', 'title': 'Latin American'}...",4.0,"{'latitude': 40.68431, 'longitude': -73.99197}","[delivery, pickup]",$$,"{'address1': '221 Smith St', 'address2': '', '...",+13479873260,(347) 987-3260,2361.433794
997,EFihj7vsRRXiF5la-UURyw,hop-kee-new-york,Hop Kee,https://s3-media1.fl.yelpcdn.com/bphoto/S32no6...,False,https://www.yelp.com/biz/hop-kee-new-york?adju...,911,"[{'alias': 'seafood', 'title': 'Seafood'}, {'a...",3.5,"{'latitude': 40.714458, 'longitude': -73.998889}",[delivery],$$,"{'address1': '21 Mott St', 'address2': '', 'ad...",+12129648365,(212) 964-8365,1074.470972
998,JwCnSW-RQ7_DxvbzWNO1hA,coletta-new-york-2,Coletta,https://s3-media2.fl.yelpcdn.com/bphoto/v8F0R1...,False,https://www.yelp.com/biz/coletta-new-york-2?ad...,234,"[{'alias': 'vegan', 'title': 'Vegan'}, {'alias...",4.5,"{'latitude': 40.74094, 'longitude': -73.98139}","[delivery, pickup]",$$$,"{'address1': '365 3rd Ave', 'address2': None, ...",+16468612889,(646) 861-2889,4089.341539
999,BdvUnXh0cI_r0znZOEPyHQ,flaming-grill-and-buffet-brooklyn,Flaming Grill & Buffet,https://s3-media3.fl.yelpcdn.com/bphoto/17BZL6...,False,https://www.yelp.com/biz/flaming-grill-and-buf...,17,"[{'alias': 'buffets', 'title': 'Buffets'}, {'a...",3.5,"{'latitude': 40.646698, 'longitude': -73.957205}",[],NaN,"{'address1': '2163 Tilden Ave', 'address2': No...",+13473654818,(347) 365-4818,7234.304060
